# Data Mining/ML 

 **2020MT10823 - Mohit Kumar Gond** <br>


#  Question 2 : MNIST

The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits used for training various image processing systems.

It has 60,000 training and 10,000 testing images 

# Imports

In [1]:
#Data import 
import pandas as pd
import numpy as np

#Data-preprocessing (removing outliners and standardization)
from sklearn.preprocessing import StandardScaler
from numpy import percentile
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from tensorflow.keras import layers

# Hyper Parameter Tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Model Impletation(Decision Tree,Random Forest,KNN classifier, Naive Bayes Classifier, SVM  and ANN)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

#K-Fold Validation for Model Training
from sklearn.model_selection import KFold

#Evaluation Metrics 
from sklearn.metrics import accuracy_score
from sklearn.metrics import  classification_report,confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_score

C:\Users\prati\AppData\Local\Temp/ipykernel_14980/842453223.py:11: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


# Data extraction and  Pre-Processing

In [3]:
mnist = fetch_openml('mnist_784')
X = mnist['data']
y=mnist['target']

## Training - Testing data split

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)
print(f"X_train : {X_train.shape}")
print(f"Y_train : {y_train.shape}")
print(f"X_test : {X_test.shape}")
print(f"Y_test : {y_test.shape}")

# 1) Decision Tree

A tree-like model of decisions and their consequences by recursively splitting the dataset based on the most significant feature at each node  of Decision Tree.
At each node of the tree, the algorithm asks a question about a feature that divides the data into two or more subsets. This process is repeated until a certain stopping criterion is met, such as when all data in a subset belongs to the same class or when a certain maximum depth or minimum number of samples per leaf node is reached.


#### **sklearn.tree.DecisionTreeClassifier**:
The corresponding **Hyper Parameters** are:
- **max_features** : Numbers of features considered for split at each node.
- **ccp_alpha** : Complexity parameter used for minimal cost-complexity pruning.
- **max_depth** : Maximum depth of the tree.
- **criterion** : Function to measure the quality of a split.
- **min_samples_split** : The minimum number of samples required to split a node.
- **min_samples_leaf** : Minimum number of samples required to be at a leaf node.

In [ ]:
# Decision Tree Classifier
dtc = DecisionTreeClassifier()

Search space for Hyperparameters Tuning is as follows:
> max_features = ["auto", "sqrt", "log2"] \
> ccp_alpha = [0.1, .01, .001] \
> max_depth = [5, 6, 7, 8, 9] \
> criterion = ['gini', 'entropy'] \

## Hyper Parameter Tuning using **Grid Search CV**:

In [ ]:
# Grid Search CV
max_features = ["auto", "sqrt", "log2"] # Number of features considered at each split
ccp_alpha = [0.1, .01, .001]
max_depth = [5, 6, 7, 8, 9] # Max height of Trees
criterion = ['gini', 'entropy']

param_grid = {'max_features': max_features,
              'ccp_alpha': ccp_alpha,
              'max_depth' : max_depth,
              'criterion' : criterion
             }

param_grid

In [ ]:
dtc_grid = GridSearchCV(estimator = dtc, param_grid = param_grid, cv = 5, verbose = 2)
dtc_grid.fit(X_train, y_train)

In [ ]:
best_param = dtc_grid.best_params_
print("Best Parameters for Decision Tree after Grid Search CV: ")
best_param
print()

#### Model Quality: 

In [ ]:
print("Accuracy Score: ")
train_accuracy = dtc_grid.score(X_train, y_train)
test_accuracy = dtc_grid.score(X_test, y_test)
print(f"Train Accuracy : {train_accuracy}")
print(f"Test Accuracy : {test_accuracy}")
print()

print("Classification Report: ")
dtc_best = DecisionTreeClassifier(ccp_alpha=best_param['ccp_alpha'], criterion=best_param['criterion'], max_depth=best_param['max_depth'], max_features=best_param['max_features'])
dtc_best.fit(X_train, y_train)
y_predict = dtc_best.predict(X_test)
print(classification_report(y_test, y_predict))
print()

### K-Fold CV on 'Best' model obtained after Grid Search CV :

In [ ]:
scores=[]
acc = []
# print(X.shape)
# print(Y.shape)
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=10,random_state=None,shuffle=False)
for train_index,test_index in kf.split(X):
    print(test_index.shape)
    print(train_index.shape)
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],Y.iloc[train_index],Y.iloc[test_index]
    dtc_best.fit(X_train,y_train)
    y_predict = dtc_best.predict(X_test)
    acc.append(accuracy_score(y_test,y_predict))
    scores.append(dtc_best.score(X_test,y_test))

print(np.mean(acc))
cross_val_score(dtc_best,X,Y,cv=10)


## Hyper Parameter Tuning using **Random Search CV**:

In [ ]:
# Random Search CV
max_features = ["auto", "sqrt", "log2"] # Number of features considered at each split
ccp_alpha = [0.1, .01, .001]
max_depth = [5, 6, 7, 8, 9] # Max height of Trees
criterion = ['gini', 'entropy']

param_grid = {'max_features': max_features,
              'ccp_alpha': ccp_alpha,
              'max_depth' : max_depth,
              'criterion' : criterion
             }

param_grid

In [ ]:
dtc_random = RandomizedSearchCV(estimator = dtc, param_distributions = param_grid, cv = 5)
dtc_random.fit(X_train, y_train)

In [ ]:
best_param = dtc_random.best_params_
print("Best Parameters for Decision Tree after Random Search CV: ")
best_param
print()

#### Model Quality

In [ ]:
print("Accuracy Score: ")
train_accuracy = dtc_random.score(X_train, y_train)
test_accuracy = dtc_random.score(X_test, y_test)
print(f"Train Accuracy : {train_accuracy}")
print(f"Test Accuracy : {test_accuracy}")
print()

print("Classification Report: ")
dtc_best = DecisionTreeClassifier(ccp_alpha=best_param['ccp_alpha'], criterion=best_param['criterion'], max_depth=best_param['max_depth'], max_features=best_param['max_features'])
dtc_best.fit(X_train, y_train)
y_predict = dtc_best.predict(X_test)
print(classification_report(y_test, y_predict))
print()

### K-Fold CV on 'Best' model obtained after Random Search CV :

In [ ]:
scores=[]
acc = []
# print(X.shape)
# print(Y.shape)
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=10,random_state=None,shuffle=False)
for train_index,test_index in kf.split(X):
    print(test_index.shape)
    print(train_index.shape)
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],Y.iloc[train_index],Y.iloc[test_index]
    dtc_best.fit(X_train,y_train)
    y_predict = dtc_best.predict(X_test)
    acc.append(accuracy_score(y_test,y_predict))
    scores.append(dtc_best.score(X_test,y_test))

print(np.mean(acc))
cross_val_score(dtc_best,X,Y,cv=10)


# 2) Random Forest

Random forest classifier is a clasifier based on multiple Decision tree classifiers which are trained on different fraction of training data. In case of classification the majority vote of those decision trees is taken as final output.


#### **sklearn.ensemble.RandomForestClassifier**:
The corresponding **Hyper Parameters** are:
- **n_estimators** : Numbers of decision trees.
- **max_features** : Size of random subsets of features to consider when splitting a node.
- **max_depth** : Maximum depth of the tree.
- **min_samples_split** : The minimum number of samples required to split a node.
- **bootstrap** : Whole dataset will be used when bootstrap=false.

In [ ]:
# Random Forest Classifier
rfc = RandomForestClassifier()

Search space for Hyperparameters Tuning is as follows:
> n_estimators = [x for x in range(25, 40, 2)] \
> max_features = ["auto", "sqrt"] \
> max_depth = [5, 6, 7] \
> min_samples_split = [2, 5, 10] \ 
> min_samples_leaf = [1, 2, 4] \
> bootstrap = [True, False] 

### Grid Search CV:

## Hyper Parameter Tuning using **Grid Search CV**:

In [ ]:
# Grid Search CV
n_estimators = [x for x in range(25, 40, 2)]
max_features = ["auto", "sqrt"]
max_depth = [5, 6, 7] 
min_samples_split = [2, 5, 10] 
min_samples_leaf = [1, 2, 4] 
bootstrap = [True, False]

param_grid = {
    "n_estimators" : n_estimators,
    "max_features" : max_features,
    "max_depth" : max_depth,
    "min_samples_split" : min_samples_split,
    "min_samples_leaf" : min_samples_leaf,
    "bootstrap" : bootstrap
}

param_grid

{'n_estimators': [25, 27, 29, 31, 33, 35, 37, 39],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [5, 6, 7],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': [1, 2, 4],
 'bootstrap': [True, False]}

In [ ]:
rfc_grid = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 3, verbose = 2, n_jobs = 4)
rfc_grid.fit(X_train, y_train)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is als

[CV] END bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   4.2s
[CV] END bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   4.2s
[CV] END bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   4.6s


/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=27; total time=   5.0s


...

[CV] END bootstrap=True, max_depth=6, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=37; total time=  10.1s


/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=6, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=37; total time=   9.2s


/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=True, max_depth=6, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=37; total time=   7.0s
[CV] END bootstrap=True, max_depth=6, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=39; total time=   7.7s
[CV] END bootstrap=True, max_depth=6, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=39; total time=   7.1s
[CV] END bootstrap=True, max_depth=6, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=39; total time=   6.8s
[CV] END bootstrap=True, max_depth=6, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   4.7s
[CV] END bootstrap=True, max_depth=6, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   4.6s
[CV] END bootstrap=True, max_depth=6, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=25; total time=   4.5s
[CV] END bootstrap=True, max_depth=6, max_features=sqrt, m

In [ ]:
best_param = rfc_grid.best_params_
print("Best Parameters for Decision Tree after Grid Search CV: ")
best_param

Best Parameters for Decision Tree after Grid Search CV: 


{'bootstrap': False,
 'max_depth': 7,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 37}

#### Model Quality: 

In [ ]:
print("Accuracy Score: ")
train_accuracy = rfc_grid.score(X_train, y_train)
test_accuracy = rfc_grid.score(X_test, y_test)
print(f"Train Accuracy : {train_accuracy}")
print(f"Test Accuracy : {test_accuracy}")
print()

rfc_best = RandomForestClassifier(bootstrap=best_param['bootstrap'], max_depth=best_param['max_depth'], max_features=best_param['max_features'], min_samples_leaf=best_param['min_samples_leaf'], min_samples_split=best_param['min_samples_split'], n_estimators=best_param['n_estimators'])
rfc_best.fit(X_train, y_train)
print("Classification Report: ")
y_predict = rfc_best.predict(X_test)
print(classification_report(y_test, y_predict))

Accuracy Score: 
Train Accuracy : 0.9306526806526807
Test Accuracy : 0.9036519036519036



/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Classification Report: 
              precision    recall  f1-score   support

           1       0.92      0.98      0.95       309
           2       0.97      0.98      0.97       131
           3       0.87      0.96      0.91       280
           4       0.73      0.63      0.68       115
           5       0.97      0.76      0.85       147
           7       0.90      0.89      0.90       305

    accuracy                           0.90      1287
   macro avg       0.89      0.87      0.88      1287
weighted avg       0.90      0.90      0.90      1287



### K-Fold CV on 'Best' model obtained after Grid Search CV :

In [ ]:
scores=[]
acc = []
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=10,random_state=None,shuffle=False)
for train_index,test_index in kf.split(X):
    # print(test_index.shape)
    # print(train_index.shape)
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],Y.iloc[train_index],Y.iloc[test_index]
    rfc_best.fit(X_train,y_train)
    y_predict = rfc_best.predict(X_test)
    acc.append(accuracy_score(y_test,y_predict))
    scores.append(rfc_best.score(X_test,y_test))

print(f"Mean Accuracy Score: {np.mean(acc)}")
print()
print("Cross Validation Score: ")
cross_val_score(rfc_best,X,Y,cv=10)

/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is als

Mean Accuracy Score: 0.8728903238893772

Cross Validation Score: 


/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is als

array([0.92391304, 0.88509317, 0.8121118 , 0.89285714, 0.90993789,
       0.85381026, 0.84292379, 0.89891135, 0.90046656, 0.82892691])

## Hyper Parameter Tuning using **Random Search CV**:

In [ ]:
# Random Search CV:
n_estimators = [x for x in range(25, 40, 2)] # Number of trees
max_features = ["auto", "sqrt"] # Number of features considered at each split
max_depth = [3, 4, 5] # Max height of Trees
min_samples_split = [2, 5, 10] # Minimun number of samples required for splitting a node
min_samples_leaf = [1, 2], 4 # Minimum number of samples required at each leaf node
bootstrap = [True, False] # Method of selecting samples

param_grid = {
    "n_estimators" : n_estimators,
    "max_features" : max_features,
    "max_depth" : max_depth,
    "min_samples_split" : min_samples_split,
    "min_samples_leaf" : min_samples_leaf,
    "bootstrap" : bootstrap
}

param_grid

{'n_estimators': [25, 27, 29, 31, 33, 35, 37, 39],
 'max_features': ['auto', 'sqrt'],
 'max_depth': [3, 4, 5],
 'min_samples_split': [2, 5, 10],
 'min_samples_leaf': ([1, 2], 4),
 'bootstrap': [True, False]}

In [ ]:
rfc_random = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 3, verbose = 2, n_jobs = 4)
rfc_random.fit(X_train, y_train)

Fitting 3 folds for each of 576 candidates, totalling 1728 fits
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=25; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=25; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=25; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=27; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=27; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=27; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=2,

/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is als

[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=10, n_estimators=37; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=10, n_estimators=39; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=10, n_estimators=39; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=10, n_estimators=33; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=10, n_estimators=39; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=10, n_estimators=33; total time=   0.0s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=[1, 2], min_samples_split=10, n_estimators=33; total time=   0.0s
[CV] END bootstrap=T

/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is als

[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=35; total time=   0.2s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=25; total time=   0.2s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=25; total time=   0.2s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=31; total time=   0.2s
[CV] END bootstrap=True, max_depth=3, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=27; total time=   0.1s


...

[CV] END bootstrap=False, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=39; total time=   0.5s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=27; total time=   0.3s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=25; total time=   0.3s
[CV] END bootstrap=False, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=37; total time=   0.4s
[CV] END bootstrap=False, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=39; total time=   0.5s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=25; total time=   0.3s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=27; total time=   0.3s
[CV] END bootstrap=False, max_depth=5, max_features=

/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=25; total time=   0.3s
[CV] END bootstrap=False, max_depth=5, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=39; total time=   0.4s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=25; total time=   0.0s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=25; total time=   0.0s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=25; total time=   0.0s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=[1, 2], min_samples_split=2, n_estimators=27; total time=   0.0s
[CV] END bootstrap=False, max_depth=5, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=29; total time=   0.4s
[CV] END bootstrap=False, max_dept

/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
864 fits failed out of a total of 1728.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
864 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/home/ravi-raj/.local/lib/python3.10/site-packages/sklearn/base.py", line 581, in _validate_params
    validate_parameter_constraints(
  File "/home

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [3, 4, 5],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': ([1, 2], 4),
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [25, 27, 29, 31, 33, 35, 37, 39]},
             verbose=2)

In [ ]:
best_param = rfc_random.best_params_
print("Best Parameters for Decision Tree after Random Search CV: ")
best_param

Best Parameters for Decision Tree after Random Search CV: 


{'bootstrap': False,
 'max_depth': 5,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 29}

#### Model Quality

In [ ]:
print("Accuracy Score: ")
train_accuracy = rfc_random.score(X_train, y_train)
test_accuracy = rfc_random.score(X_test, y_test)
print(f"Train Accuracy : {train_accuracy}")
print(f"Test Accuracy : {test_accuracy}")
print()

print("Classification Report: ")
rfc_best = RandomForestClassifier(bootstrap=best_param['bootstrap'], max_depth=best_param['max_depth'], max_features=best_param['max_features'], min_samples_leaf=best_param['min_samples_leaf'], min_samples_split=best_param['min_samples_split'], n_estimators=best_param['n_estimators'])
rfc_best.fit(X_train, y_train)
y_predict = rfc_best.predict(X_test)
print(classification_report(y_test, y_predict))
print()

Accuracy Score: 
Train Accuracy : 0.882078729281768
Test Accuracy : 0.8522550544323484

Classification Report: 
              precision    recall  f1-score   support

           1       0.90      0.98      0.94       367
           2       0.83      1.00      0.91        15
           3       0.86      0.88      0.87       104
           4       0.34      0.52      0.42        21
           5       1.00      0.38      0.55        68
           7       0.76      0.66      0.71        68

    accuracy                           0.85       643
   macro avg       0.78      0.74      0.73       643
weighted avg       0.87      0.85      0.84       643




### K-Fold CV on 'Best' model obtained after Random Search CV :

In [ ]:
scores=[]
acc = []
# print(X.shape)
# print(Y.shape)
from sklearn.model_selection import cross_val_score
kf = KFold(n_splits=10,random_state=None,shuffle=False)
for train_index,test_index in kf.split(X):
    # print(test_index.shape)
    # print(train_index.shape)
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],Y.iloc[train_index],Y.iloc[test_index]
    rfc_best.fit(X_train,y_train)
    y_predict = rfc_best.predict(X_test)
    acc.append(accuracy_score(y_test,y_predict))
    scores.append(rfc_best.score(X_test,y_test))

print(f"Mean Accuracy Score: {np.mean(acc)}")
print()
print("Cross Validation Score: ")
cross_val_score(rfc_best,X,Y,cv=10)

Mean Accuracy Score: 0.8537752480125189

Cross Validation Score: 


array([0.90838509, 0.86645963, 0.80279503, 0.88043478, 0.87111801,
       0.82581649, 0.80404355, 0.87869362, 0.88491446, 0.7807154 ])

# 3) Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb= GaussianNB()


In [ ]:
from sklearn.model_selection import GridSearchCV
params_grid = {'var_smoothing': np.logspace(0,-9, num=100)}
grid = GridSearchCV(gnb,params_grid, refit = True,verbose=3)
grid.fit(X_train,y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5] END .................var_smoothing=1.0;, score=0.764 total time=   0.0s
[CV 2/5] END .................var_smoothing=1.0;, score=0.743 total time=   0.0s
[CV 3/5] END .................var_smoothing=1.0;, score=0.743 total time=   0.0s
[CV 4/5] END .................var_smoothing=1.0;, score=0.686 total time=   0.0s
[CV 5/5] END .................var_smoothing=1.0;, score=0.729 total time=   0.0s
[CV 1/5] END ..var_smoothing=0.8111308307896871;, score=0.764 total time=   0.0s
[CV 2/5] END ..var_smoothing=0.8111308307896871;, score=0.750 total time=   0.0s
[CV 3/5] END ..var_smoothing=0.8111308307896871;, score=0.743 total time=   0.0s
[CV 4/5] END ..var_smoothing=0.8111308307896871;, score=0.693 total time=   0.0s
[CV 5/5] END ..var_smoothing=0.8111308307896871;, score=0.743 total time=   0.0s
[CV 1/5] END ...var_smoothing=0.657933224657568;, score=0.771 total time=   0.0s
[CV 2/5] END ...var_smoothing=0.65793322465756

[CV 2/5] END var_smoothing=0.012328467394420659;, score=0.736 total time=   0.0s
[CV 3/5] END var_smoothing=0.012328467394420659;, score=0.736 total time=   0.0s
[CV 4/5] END var_smoothing=0.012328467394420659;, score=0.757 total time=   0.0s
[CV 5/5] END var_smoothing=0.012328467394420659;, score=0.671 total time=   0.0s
[CV 1/5] END ................var_smoothing=0.01;, score=0.764 total time=   0.0s
[CV 2/5] END ................var_smoothing=0.01;, score=0.729 total time=   0.0s
[CV 3/5] END ................var_smoothing=0.01;, score=0.736 total time=   0.0s
[CV 4/5] END ................var_smoothing=0.01;, score=0.757 total time=   0.0s
[CV 5/5] END ................var_smoothing=0.01;, score=0.671 total time=   0.0s
[CV 1/5] END var_smoothing=0.008111308307896872;, score=0.757 total time=   0.0s
[CV 2/5] END var_smoothing=0.008111308307896872;, score=0.721 total time=   0.0s
[CV 3/5] END var_smoothing=0.008111308307896872;, score=0.729 total time=   0.0s
[CV 4/5] END var_smoothing=0

[CV 5/5] END var_smoothing=0.0001873817422860383;, score=0.536 total time=   0.0s
[CV 1/5] END var_smoothing=0.0001519911082952933;, score=0.779 total time=   0.0s
[CV 2/5] END var_smoothing=0.0001519911082952933;, score=0.657 total time=   0.0s
[CV 3/5] END var_smoothing=0.0001519911082952933;, score=0.636 total time=   0.0s
[CV 4/5] END var_smoothing=0.0001519911082952933;, score=0.729 total time=   0.0s
[CV 5/5] END var_smoothing=0.0001519911082952933;, score=0.536 total time=   0.0s
[CV 1/5] END var_smoothing=0.0001232846739442066;, score=0.779 total time=   0.0s
[CV 2/5] END var_smoothing=0.0001232846739442066;, score=0.657 total time=   0.0s
[CV 3/5] END var_smoothing=0.0001232846739442066;, score=0.636 total time=   0.0s
[CV 4/5] END var_smoothing=0.0001232846739442066;, score=0.721 total time=   0.0s
[CV 5/5] END var_smoothing=0.0001232846739442066;, score=0.536 total time=   0.0s
[CV 1/5] END ..............var_smoothing=0.0001;, score=0.779 total time=   0.0s
[CV 2/5] END ....

[CV 1/5] END var_smoothing=2.310129700083158e-06;, score=0.750 total time=   0.0s
[CV 2/5] END var_smoothing=2.310129700083158e-06;, score=0.629 total time=   0.0s
[CV 3/5] END var_smoothing=2.310129700083158e-06;, score=0.600 total time=   0.0s
[CV 4/5] END var_smoothing=2.310129700083158e-06;, score=0.686 total time=   0.0s
[CV 5/5] END var_smoothing=2.310129700083158e-06;, score=0.507 total time=   0.0s
[CV 1/5] END var_smoothing=1.873817422860383e-06;, score=0.743 total time=   0.0s
[CV 2/5] END var_smoothing=1.873817422860383e-06;, score=0.629 total time=   0.0s
[CV 3/5] END var_smoothing=1.873817422860383e-06;, score=0.600 total time=   0.0s
[CV 4/5] END var_smoothing=1.873817422860383e-06;, score=0.686 total time=   0.0s
[CV 5/5] END var_smoothing=1.873817422860383e-06;, score=0.507 total time=   0.0s
[CV 1/5] END var_smoothing=1.519911082952933e-06;, score=0.743 total time=   0.0s
[CV 2/5] END var_smoothing=1.519911082952933e-06;, score=0.629 total time=   0.0s
[CV 3/5] END var

[CV 4/5] END var_smoothing=3.5111917342151277e-08;, score=0.671 total time=   0.0s
[CV 5/5] END var_smoothing=3.5111917342151277e-08;, score=0.493 total time=   0.0s
[CV 1/5] END var_smoothing=2.848035868435799e-08;, score=0.721 total time=   0.0s
[CV 2/5] END var_smoothing=2.848035868435799e-08;, score=0.636 total time=   0.0s
[CV 3/5] END var_smoothing=2.848035868435799e-08;, score=0.593 total time=   0.0s
[CV 4/5] END var_smoothing=2.848035868435799e-08;, score=0.671 total time=   0.0s
[CV 5/5] END var_smoothing=2.848035868435799e-08;, score=0.493 total time=   0.0s
[CV 1/5] END var_smoothing=2.310129700083158e-08;, score=0.721 total time=   0.0s
[CV 2/5] END var_smoothing=2.310129700083158e-08;, score=0.636 total time=   0.0s
[CV 3/5] END var_smoothing=2.310129700083158e-08;, score=0.593 total time=   0.0s
[CV 4/5] END var_smoothing=2.310129700083158e-08;, score=0.671 total time=   0.0s
[CV 5/5] END var_smoothing=2.310129700083158e-08;, score=0.493 total time=   0.0s
[CV 1/5] END v

GridSearchCV(estimator=GaussianNB(),
             param_grid={'var_smoothing': array([1.00000000e+00, 8.11130831e-01, 6.57933225e-01, 5.33669923e-01,
       4.32876128e-01, 3.51119173e-01, 2.84803587e-01, 2.31012970e-01,
       1.87381742e-01, 1.51991108e-01, 1.23284674e-01, 1.00000000e-01,
       8.11130831e-02, 6.57933225e-02, 5.33669923e-02, 4.32876128e-02,
       3.51119173e-02, 2.84803587e-02, 2.3101297...
       1.23284674e-07, 1.00000000e-07, 8.11130831e-08, 6.57933225e-08,
       5.33669923e-08, 4.32876128e-08, 3.51119173e-08, 2.84803587e-08,
       2.31012970e-08, 1.87381742e-08, 1.51991108e-08, 1.23284674e-08,
       1.00000000e-08, 8.11130831e-09, 6.57933225e-09, 5.33669923e-09,
       4.32876128e-09, 3.51119173e-09, 2.84803587e-09, 2.31012970e-09,
       1.87381742e-09, 1.51991108e-09, 1.23284674e-09, 1.00000000e-09])},
             verbose=3)

In [ ]:
grid.best_score_

0.7757142857142857

In [ ]:
grid.best_params_

{'var_smoothing': 0.15199110829529336}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gridr = {'var_smoothing': np.logspace(0,-9, num=100)}
gridr = RandomizedSearchCV(gnb,gridr, refit = True,verbose=3)
gridr=gridr.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END var_smoothing=5.336699231206313e-06;, score=0.629 total time=   2.1s
[CV 2/5] END var_smoothing=5.336699231206313e-06;, score=0.619 total time=   1.6s
[CV 3/5] END var_smoothing=5.336699231206313e-06;, score=0.641 total time=   1.6s
[CV 4/5] END var_smoothing=5.336699231206313e-06;, score=0.630 total time=   1.6s
[CV 5/5] END var_smoothing=5.336699231206313e-06;, score=0.626 total time=   1.6s
[CV 1/5] END ..var_smoothing=0.8111308307896871;, score=0.731 total time=   1.5s
[CV 2/5] END ..var_smoothing=0.8111308307896871;, score=0.738 total time=   1.7s
[CV 3/5] END ..var_smoothing=0.8111308307896871;, score=0.743 total time=   1.7s
[CV 4/5] END ..var_smoothing=0.8111308307896871;, score=0.733 total time=   1.6s
[CV 5/5] END ..var_smoothing=0.8111308307896871;, score=0.741 total time=   1.6s
[CV 1/5] END var_smoothing=3.511191734215127e-05;, score=0.659 total time=   1.6s
[CV 2/5] END var_smoothing=3.5111917342151

In [ ]:
gridr.best_score_

0.8211798711810798

In [ ]:
gridr.best_params_

{'force_alpha': True, 'fit_prior': True, 'alpha': 0.01}

In [ ]:
gnb= GaussianNB(var_smoothing= 0.15199110829529336)
kf = KFold(n_splits=3,random_state=None,shuffle=False)
scores=[]


In [ ]:
for train_index,test_index in kf.split(X):
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
    gnb.fit(X_train,y_train)
    y_pred=gnb.predict(X_test)
    scores.append(accuracy_score(y_test,y_pred)) 
np.mean(scores)  

0.7994286270601965

In [ ]:
from sklearn.metrics import  classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2316
           1       0.72      0.97      0.83      2563
           2       0.92      0.76      0.83      2371
           3       0.82      0.79      0.80      2381
           4       0.86      0.68      0.76      2299
           5       0.90      0.64      0.75      2117
           6       0.85      0.92      0.89      2251
           7       0.94      0.83      0.88      2454
           8       0.72      0.72      0.72      2311
           9       0.62      0.88      0.73      2270

    accuracy                           0.81     23333
   macro avg       0.83      0.81      0.81     23333
weighted avg       0.83      0.81      0.81     23333



In [ ]:
print(confusion_matrix(y_test,y_pred))

[[2090    4    4    5    2   40   60    1   95   15]
 [   0 2493    4    8    0    1   11    1   41    4]
 [  39  120 1797   52   29    9  129   31  153   12]
 [  19  124   68 1870    2   40   40   16  109   93]
 [   4   46    5    0 1561    4   35    2   33  609]
 [  51  108   14  233   48 1348   60   22  105  128]
 [  19   76   18    0   17   25 2071    0   23    2]
 [  11  119   11   13   40    0    1 2027   41  191]
 [  16  297   22   76   22   29   20   10 1666  153]
 [  16   67    2   21   90    3    0   39   44 1988]]


## Multinomial Naive bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb=MultinomialNB()

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0],'force_alpha' : [True,False],'fit_prior':[True,False],}
grid = GridSearchCV(mnb,param_grid, refit = True,verbose=3)
grid.fit(X_train,y_train)


Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.824 total time=   2.3s
[CV 2/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.817 total time=   0.7s
[CV 3/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.825 total time=   0.8s
[CV 4/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.820 total time=   0.5s
[CV 5/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.820 total time=   0.8s
[CV 1/5] END alpha=0.01, fit_prior=True, force_alpha=False;, score=0.824 total time=   0.6s
[CV 2/5] END alpha=0.01, fit_prior=True, force_alpha=False;, score=0.817 total time=   0.8s
[CV 3/5] END alpha=0.01, fit_prior=True, force_alpha=False;, score=0.825 total time=   0.5s
[CV 4/5] END alpha=0.01, fit_prior=True, force_alpha=False;, score=0.820 total time=   0.8s
[CV 5/5] END alpha=0.01, fit_prior=True, force_alpha=False;, score=0.820 total time=   0.5s
[CV 1/5] END alpha=0.01,

GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.01, 0.1, 1.0, 10.0],
                         'fit_prior': [True, False],
                         'force_alpha': [True, False]},
             verbose=3)

In [ ]:
grid.best_score_

0.8211798711810798

In [ ]:
grid.best_params_

{'alpha': 0.01, 'fit_prior': True, 'force_alpha': True}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {'alpha': [0.01, 0.1, 1.0, 10.0],'force_alpha' : [True,False],'fit_prior':[True,False],}
gridr = RandomizedSearchCV(mnb,param_grid, refit = True,verbose=3)
gridr.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.824 total time=   0.4s
[CV 2/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.817 total time=   0.5s
[CV 3/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.825 total time=   0.4s
[CV 4/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.820 total time=   0.4s
[CV 5/5] END alpha=0.01, fit_prior=True, force_alpha=True;, score=0.820 total time=   0.2s
[CV 1/5] END alpha=0.01, fit_prior=False, force_alpha=False;, score=0.824 total time=   0.3s
[CV 2/5] END alpha=0.01, fit_prior=False, force_alpha=False;, score=0.817 total time=   0.3s
[CV 3/5] END alpha=0.01, fit_prior=False, force_alpha=False;, score=0.825 total time=   0.3s
[CV 4/5] END alpha=0.01, fit_prior=False, force_alpha=False;, score=0.820 total time=   0.3s
[CV 5/5] END alpha=0.01, fit_prior=False, force_alpha=False;, score=0.820 total time=   0.3s
[CV 1/5] END alpha=

RandomizedSearchCV(estimator=MultinomialNB(),
                   param_distributions={'alpha': [0.01, 0.1, 1.0, 10.0],
                                        'fit_prior': [True, False],
                                        'force_alpha': [True, False]},
                   verbose=3)

In [ ]:
gridr.best_score_

0.8211798711810798

In [ ]:
gridr.best_params_

{'force_alpha': True, 'fit_prior': True, 'alpha': 0.01}

In [ ]:
mnb=MultinomialNB(alpha= 0.01, fit_prior= True, force_alpha= True)
mnb.fit(X_train, y_train)
y_pred2=mnb.predict(X_test)
y_pred_train=mnb.predict(X_train)
acc1=accuracy_score(y_train,y_pred_train)
print(f"accuracy on train with best param by randomsearch {acc1:0.4f}")
acc=accuracy_score(y_test,y_pred)
print(f"accuracy on test with best param by randomSearch  {acc:0.4f} ")

accuracy on train with best param by randomsearch 0.8228
accuracy on test with best param by randomSearch  0.8105 


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3,random_state=None,shuffle=False)
scores=[]

In [ ]:
for train_index,test_index in kf.split(X):
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
    mnb.fit(X_train,y_train)
    y_pred=mnb.predict(X_test)
    scores.append(accuracy_score(y_test,y_pred))

np.mean(scores)

0.8257286562541877

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.92      2316
           1       0.89      0.94      0.92      2563
           2       0.89      0.84      0.86      2371
           3       0.81      0.83      0.82      2381
           4       0.85      0.76      0.80      2299
           5       0.88      0.68      0.77      2117
           6       0.89      0.92      0.90      2251
           7       0.95      0.83      0.89      2454
           8       0.69      0.80      0.74      2311
           9       0.69      0.85      0.76      2270

    accuracy                           0.84     23333
   macro avg       0.85      0.84      0.84     23333
weighted avg       0.85      0.84      0.84     23333



In [ ]:
print(confusion_matrix(y_test,y_pred))

[[2115    1    8    9    5   20   34    1  119    4]
 [   0 2415   17   16    0    8   14    0   88    5]
 [  31   32 1995   40   24    6   82   24  133    4]
 [  17   30   97 1984    2   49   26   15   95   66]
 [   5   11   10    1 1742    4   36    3   80  407]
 [  53   14   22  247   37 1440   51   10  159   84]
 [  26   34   34    1   12   35 2070    0   38    1]
 [  16   47   21   16   47    1    2 2045   74  185]
 [  16  101   33  103   27   60   15   13 1851   92]
 [  13   25    7   29  147   16    0   51   60 1922]]


# 4) KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_neighbors' : [3,4,5,6,7,8,9,10,11,12,13,14,15],
              'weights': ['uniform', 'distance'],
              'algorithm' : ['auto', 'ball_tree','kd_tree', 'brute'],
              'p':[1],'leaf_size':[12,20,30,40]}
grid = GridSearchCV(neigh,param_grid, refit = True,verbose=3)
grid=grid.fit(X_train,y_train)

Fitting 5 folds for each of 416 candidates, totalling 2080 fits
[CV 1/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=uniform;, score=0.903 total time=   1.0s
[CV 2/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=uniform;, score=0.887 total time=   0.5s
[CV 3/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=uniform;, score=0.896 total time=   0.6s
[CV 4/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=uniform;, score=0.917 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=uniform;, score=0.913 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=distance;, score=0.901 total time=   0.4s
[CV 2/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=distance;, score=0.894 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=12, n_neighbors=3, p=1, weights=distance;, score=0.904 total time=   0.4s
[CV 4/5] END algorithm=auto, leaf_siz

[CV 4/5] END algorithm=auto, leaf_size=12, n_neighbors=10, p=1, weights=uniform;, score=0.917 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=12, n_neighbors=10, p=1, weights=uniform;, score=0.906 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=12, n_neighbors=10, p=1, weights=distance;, score=0.890 total time=   0.4s
[CV 2/5] END algorithm=auto, leaf_size=12, n_neighbors=10, p=1, weights=distance;, score=0.886 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=12, n_neighbors=10, p=1, weights=distance;, score=0.896 total time=   0.4s
[CV 4/5] END algorithm=auto, leaf_size=12, n_neighbors=10, p=1, weights=distance;, score=0.917 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=12, n_neighbors=10, p=1, weights=distance;, score=0.914 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=12, n_neighbors=11, p=1, weights=uniform;, score=0.887 total time=   0.4s
[CV 2/5] END algorithm=auto, leaf_size=12, n_neighbors=11, p=1, weights=uniform;, score=0.8

[CV 2/5] END algorithm=auto, leaf_size=20, n_neighbors=4, p=1, weights=distance;, score=0.893 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=20, n_neighbors=4, p=1, weights=distance;, score=0.910 total time=   0.4s
[CV 4/5] END algorithm=auto, leaf_size=20, n_neighbors=4, p=1, weights=distance;, score=0.927 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=20, n_neighbors=4, p=1, weights=distance;, score=0.920 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=20, n_neighbors=5, p=1, weights=uniform;, score=0.890 total time=   0.4s
[CV 2/5] END algorithm=auto, leaf_size=20, n_neighbors=5, p=1, weights=uniform;, score=0.884 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=20, n_neighbors=5, p=1, weights=uniform;, score=0.904 total time=   0.4s
[CV 4/5] END algorithm=auto, leaf_size=20, n_neighbors=5, p=1, weights=uniform;, score=0.926 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=20, n_neighbors=5, p=1, weights=uniform;, score=0.909 total t

[CV 5/5] END algorithm=auto, leaf_size=20, n_neighbors=11, p=1, weights=distance;, score=0.904 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=uniform;, score=0.887 total time=   0.4s
[CV 2/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=uniform;, score=0.871 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=uniform;, score=0.889 total time=   0.4s
[CV 4/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=uniform;, score=0.903 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=uniform;, score=0.897 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=distance;, score=0.891 total time=   0.4s
[CV 2/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=distance;, score=0.880 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=20, n_neighbors=12, p=1, weights=distance;, score=0.90

[CV 3/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=uniform;, score=0.897 total time=   0.4s
[CV 4/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=uniform;, score=0.921 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=uniform;, score=0.907 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=distance;, score=0.901 total time=   0.4s
[CV 2/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=distance;, score=0.889 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=distance;, score=0.907 total time=   0.4s
[CV 4/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=distance;, score=0.930 total time=   0.4s
[CV 5/5] END algorithm=auto, leaf_size=30, n_neighbors=6, p=1, weights=distance;, score=0.923 total time=   0.4s
[CV 1/5] END algorithm=auto, leaf_size=30, n_neighbors=7, p=1, weights=uniform;, score=0.889 total 

...

[CV 2/5] END algorithm=brute, leaf_size=20, n_neighbors=12, p=1, weights=distance;, score=0.880 total time=   0.4s
[CV 3/5] END algorithm=brute, leaf_size=20, n_neighbors=12, p=1, weights=distance;, score=0.900 total time=   0.5s
[CV 4/5] END algorithm=brute, leaf_size=20, n_neighbors=12, p=1, weights=distance;, score=0.911 total time=   0.4s
[CV 5/5] END algorithm=brute, leaf_size=20, n_neighbors=12, p=1, weights=distance;, score=0.909 total time=   0.4s
[CV 1/5] END algorithm=brute, leaf_size=20, n_neighbors=13, p=1, weights=uniform;, score=0.886 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=20, n_neighbors=13, p=1, weights=uniform;, score=0.877 total time=   0.5s
[CV 3/5] END algorithm=brute, leaf_size=20, n_neighbors=13, p=1, weights=uniform;, score=0.884 total time=   0.5s
[CV 4/5] END algorithm=brute, leaf_size=20, n_neighbors=13, p=1, weights=uniform;, score=0.900 total time=   0.5s
[CV 5/5] END algorithm=brute, leaf_size=20, n_neighbors=13, p=1, weights=uniform;, s

[CV 4/5] END algorithm=brute, leaf_size=30, n_neighbors=6, p=1, weights=distance;, score=0.930 total time=   0.4s
[CV 5/5] END algorithm=brute, leaf_size=30, n_neighbors=6, p=1, weights=distance;, score=0.923 total time=   0.4s
[CV 1/5] END algorithm=brute, leaf_size=30, n_neighbors=7, p=1, weights=uniform;, score=0.889 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=30, n_neighbors=7, p=1, weights=uniform;, score=0.879 total time=   0.4s
[CV 3/5] END algorithm=brute, leaf_size=30, n_neighbors=7, p=1, weights=uniform;, score=0.899 total time=   0.4s
[CV 4/5] END algorithm=brute, leaf_size=30, n_neighbors=7, p=1, weights=uniform;, score=0.923 total time=   0.4s
[CV 5/5] END algorithm=brute, leaf_size=30, n_neighbors=7, p=1, weights=uniform;, score=0.916 total time=   0.4s
[CV 1/5] END algorithm=brute, leaf_size=30, n_neighbors=7, p=1, weights=distance;, score=0.901 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=30, n_neighbors=7, p=1, weights=distance;, score=0.88

[CV 1/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=uniform;, score=0.894 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=uniform;, score=0.871 total time=   0.4s
[CV 3/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=uniform;, score=0.881 total time=   0.4s
[CV 4/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=uniform;, score=0.903 total time=   0.5s
[CV 5/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=uniform;, score=0.903 total time=   0.4s
[CV 1/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=distance;, score=0.896 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=distance;, score=0.877 total time=   0.4s
[CV 3/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=distance;, score=0.890 total time=   0.4s
[CV 4/5] END algorithm=brute, leaf_size=30, n_neighbors=14, p=1, weights=distance;, s

[CV 4/5] END algorithm=brute, leaf_size=40, n_neighbors=8, p=1, weights=uniform;, score=0.920 total time=   0.4s
[CV 5/5] END algorithm=brute, leaf_size=40, n_neighbors=8, p=1, weights=uniform;, score=0.911 total time=   0.4s
[CV 1/5] END algorithm=brute, leaf_size=40, n_neighbors=8, p=1, weights=distance;, score=0.897 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=40, n_neighbors=8, p=1, weights=distance;, score=0.890 total time=   0.4s
[CV 3/5] END algorithm=brute, leaf_size=40, n_neighbors=8, p=1, weights=distance;, score=0.897 total time=   0.4s
[CV 4/5] END algorithm=brute, leaf_size=40, n_neighbors=8, p=1, weights=distance;, score=0.917 total time=   0.4s
[CV 5/5] END algorithm=brute, leaf_size=40, n_neighbors=8, p=1, weights=distance;, score=0.921 total time=   0.4s
[CV 1/5] END algorithm=brute, leaf_size=40, n_neighbors=9, p=1, weights=uniform;, score=0.886 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=40, n_neighbors=9, p=1, weights=uniform;, score=0.8

[CV 1/5] END algorithm=brute, leaf_size=40, n_neighbors=15, p=1, weights=distance;, score=0.887 total time=   0.4s
[CV 2/5] END algorithm=brute, leaf_size=40, n_neighbors=15, p=1, weights=distance;, score=0.876 total time=   0.4s
[CV 3/5] END algorithm=brute, leaf_size=40, n_neighbors=15, p=1, weights=distance;, score=0.889 total time=   0.4s
[CV 4/5] END algorithm=brute, leaf_size=40, n_neighbors=15, p=1, weights=distance;, score=0.904 total time=   0.4s
[CV 5/5] END algorithm=brute, leaf_size=40, n_neighbors=15, p=1, weights=distance;, score=0.901 total time=   0.4s


In [ ]:
grid.best_estimator_

KNeighborsClassifier(leaf_size=12, n_neighbors=4, p=1, weights='distance')

In [ ]:
grid.best_params_

{'algorithm': 'auto',
 'leaf_size': 12,
 'n_neighbors': 4,
 'p': 1,
 'weights': 'distance'}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {'n_neighbors' : [3,4,5,6,7,8,9,10,11,12,13,14,15],
              'weights': ['uniform', 'distance'],
              'algorithm' : ['auto', 'ball_tree','kd_tree', 'brute'],
              'p':[1],'leaf_size':[12,20,30,40]}
gridr = RandomizedSearchCV(neigh,param_grid, refit = True,verbose=3)
gridr=gridr.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END algorithm=kd_tree, leaf_size=12, n_neighbors=5, p=1, weights=distance;, score=0.896 total time=   3.3s
[CV 2/5] END algorithm=kd_tree, leaf_size=12, n_neighbors=5, p=1, weights=distance;, score=0.889 total time=   3.2s
[CV 3/5] END algorithm=kd_tree, leaf_size=12, n_neighbors=5, p=1, weights=distance;, score=0.906 total time=   3.2s
[CV 4/5] END algorithm=kd_tree, leaf_size=12, n_neighbors=5, p=1, weights=distance;, score=0.927 total time=   2.4s
[CV 5/5] END algorithm=kd_tree, leaf_size=12, n_neighbors=5, p=1, weights=distance;, score=0.917 total time=   2.2s
[CV 1/5] END algorithm=auto, leaf_size=40, n_neighbors=12, p=1, weights=distance;, score=0.891 total time=   0.3s
[CV 2/5] END algorithm=auto, leaf_size=40, n_neighbors=12, p=1, weights=distance;, score=0.880 total time=   0.4s
[CV 3/5] END algorithm=auto, leaf_size=40, n_neighbors=12, p=1, weights=distance;, score=0.900 total time=   0.4s
[CV 4/5] END algo

In [ ]:
print(gridr.best_params_)
params= grid.best_params_

{'weights': 'distance', 'p': 1, 'n_neighbors': 5, 'leaf_size': 12, 'algorithm': 'kd_tree'}


In [ ]:
gridr.best_score_

0.9068571428571428

In [ ]:
neigh=neigh2 = KNeighborsClassifier(n_neighbors=params['n_neighbors'],weights=params['weights'],algorithm=params['algorithm'],leaf_size=params['leaf_size'],p=params['p'])

In [ ]:

neigh.fit(X_train, y_train)
y_pred=neigh.predict(X_test)
acc=accuracy_score(y_test,y_pred)
print(f"accuracy with simple split is {acc:0.4f} ")

accuracy with simple split is 0.9160 


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=3,random_state=None,shuffle=False)
scores=[]

In [ ]:
for train_index,test_index in kf.split(X):
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
    neigh.fit(X_train,y_train)
    y_pred=neigh.predict(X_test)
    scores.append(accuracy_score(y_test,y_pred))

In [ ]:
np.mean(scores)

# 5) Support Vector Machine

## Hyper-parameter Tuning 

Three common approches used for hyper parameter tuning: 
1. Grid Search CV
2. Random Search CV
3. Bayes Search CV

## Randomized search CV

In [6]:
param_grid = {'C' : [0.1, 1, 10],'gamma':[1, 0.1, 0.01],'kernel':['rbf','poly','sigmoid','linear']}

randgrid = RandomizedSearchCV(SVC(),param_grid, refit = True,verbose=3)

randgrid.fit(X_search,y_search)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.115 total time=  47.7s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.114 total time=  48.1s
[CV 3/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.114 total time=  37.5s
[CV 4/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.114 total time=  53.7s
[CV 5/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.114 total time=  48.2s
[CV 1/5] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.115 total time=  40.0s
[CV 2/5] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.114 total time=  31.6s
[CV 3/5] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.114 total time=  27.1s
[CV 4/5] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.114 total time=  37.8s
[CV 5/5] END ...C=1, gamma=0.01, kernel=sigmoid;, score=0.114 total time=  30.4s
[CV 1/5] END ......C=10, gamma=0.1, kernel=poly;, score=0.939 total time=   6.1s
[CV 2/5] END ......C=10, gamma=0.1, kernel=poly;

RandomizedSearchCV(estimator=SVC(),
                   param_distributions={'C': [0.1, 1, 10],
                                        'gamma': [1, 0.1, 0.01],
                                        'kernel': ['rbf', 'poly', 'sigmoid',
                                                   'linear']},
                   verbose=3)

In [8]:
best_param = randgrid.best_params_

## GridSearchCV

In [ ]:
param_grid = {'C' : [0.1, 1, 10],'gamma':[1, 0.1, 0.01],'kernel':['rbf','poly','sigmoid','linear']}

grid = GridSearchCV(SVC(),param_grid, refit = True,verbose=3)

grid.fit(X_search,y_search)


## K-fold validation

In [ ]:

scores=[]
acc = []
svm = SVC()
kf = KFold(n_splits=10,random_state=None,shuffle=False)
for train_index,test_index in kf.split(X):
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
    svm.fit(X_train,y_train)
    y_pred = svm.predict(X_test)
    scores.append(svm.score(X_test,y_test))
    acc.append(accuracy_score(y_test,y_pred))

0.9514285714285714 is the accuracy score.
0.9514285714285714 is the cross val_score 



In [ ]:
classification_report(y_test,y_pred)

 precision    recall  f1-score   support

           0       0.97      0.99      0.98       361
           1       0.97      0.98      0.97       398
           2       0.94      0.94      0.94       350
           3       0.97      0.92      0.94       333
           4       0.94      0.95      0.95       345
           5       0.95      0.94      0.95       314
           6       0.96      0.97      0.97       343
           7       0.96      0.95      0.96       349
           8       0.93      0.92      0.93       344
           9       0.93      0.94      0.94       363

    accuracy                           0.95      3500
   macro avg       0.95      0.95      0.95      3500
weighted avg       0.95      0.95      0.95      3500


# 6) Artificial Neural Networks

## Hyper-parameter Tuning 

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 75)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='softmax'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss=keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=5,
    overwrite=True,
    executions_per_trial=3,
    directory='project',
    project_name='mnist')

In [ ]:
tuner.search(X_search, y_search,
             epochs=5,
             validation_data=(X_test, y_test))

In [ ]:
nnclassifier  = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(512,352,224,128,448,256,96,128,416,96,96,416,480,128,320,480,480,192,160,128,512,96,448,480,256,64,256,256,416,96,64,128,480),alpha=0.0001, random_state=1)
nnclassifier.max_iter=1000
nnclassifier.fit(X_train,y_train)
y_predict = nnclassifier.predict(X_test)
print(classification_report(y_test,y_predict))

K-fold validation

In [ ]:
scores=[]
nn = MLPClassifier()
kf = KFold(n_splits=10,random_state=None,shuffle=False)
for train_index,test_index in kf.split(X):
    X_train,X_test,y_train,y_test=X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
    nn.fit(X_train,y_train)
    y_pred=nn.predict(X_test)
    scores.append(accuracy_score(y_test,y_pred))
    
np.mean(scores)